# Counting Tools
## How to Count Words in Python and Pandas

This notebook gives a brief introduction to counting words in Python and Pandas. While word counting is the most basic form of text mining, it still involves a number of interpretative steps. To help you understand our choices in our article, we are providing this notebook as a supplementary resource and introduction to these tradeoffs. We are not attempting to be comprehensive here, but rather to give you a sense of the kinds of decisions that go into word counting.

For additional resources, see William J. Turkel and Adam Crymble, "Counting Word Frequencies with Python," *Programming Historian* 1 (2012), https://doi.org/10.46430/phen0003 and Megan S. Kane, "Corpus Analysis with spaCy," *Programming Historian* 12 (2023), https://doi.org/10.46430/phen0113.

### Importing Relevant Libraries and Create Shared Functions

In [1]:
import pandas as pd
import re
import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
## If you haven't downloaded the NLTK data sets yet, do so:
def download_nltk_data_if_needed(packages):
    for package in packages:
        try:
            nltk.data.find(package)
        except LookupError:
            nltk.download(package)

download_nltk_data_if_needed(['tokenizers/punkt', 'corpora/stopwords', 'wordnet'])

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/zleblanc/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
def color_cells(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, `'color: green'` for positive strings.
    """
    color = 'red' if val > 0 else 'blue'
    return 'color: %s' % color

def make_pretty(styler, subset_columns):
    styler.applymap(color_cells, subset=subset_columns)
    return styler

### Create Example Dataset

Given our article's focus on DH tools, we are creating a dataset that explores how we count the word `tool`.

In [8]:
example_tool_data = pd.DataFrame({'example_text': ['digital tools', 'Critical Tool Studies', 'Footstool', 'DH TOOLSETS', 'tooling up']})
example_tool_data

,example_text
0,digital tools
1,Critical Tool Studies
2,Footstool
3,DH TOOLSETS
4,tooling up


### Word Counting Methods

#### Method 1: String Matching

Computers are very good at counting things, but also very literal. The simplest way to count words is to tell the computer to look for the word you want to count. This is called "string matching." In Pandas, we can do this with the `str.count()` method.

In [9]:
example_tool_data['string_matching'] = example_tool_data['example_text'].str.count('tool')
example_tool_data.style.pipe(make_pretty, subset_columns=['string_matching']) 

,example_text,string_matching
0,digital tools,1
1,Critical Tool Studies,0
2,Footstool,1
3,DH TOOLSETS,0
4,tooling up,1


From this quick example, we can see that three of our examples are counted correctly, but two are not (`Critical tool Studies` and `DH TOOLSETS`). This is because the `str.count()` method is case sensitive, and those examples are capitalized and uppercase, respectively.

In [10]:
example_tool_data['string_matching'] = example_tool_data['example_text'].str.count('tool|Tool|TOOL')
example_tool_data.style.pipe(make_pretty, subset_columns=['string_matching']) 

,example_text,string_matching
0,digital tools,1
1,Critical Tool Studies,1
2,Footstool,1
3,DH TOOLSETS,1
4,tooling up,1


Here we used a simple `OR` pipe operator to count all the versions of tools, which has worked. But we are currently counting `Footstool` a word that contains `tool` but not an instance of `tool`. Consequently, straight string matching is slightly too permissive for our purposes.

#### Method 2: Tokenization

Rather than counting strings, we can count words. This is called "tokenization." Tokenization is the process of breaking up a string into smaller units, called tokens. In this case, we want to break up our string into words. We can do this with the `str.split()` method. Tokenization is very language-specific, but since our data is in English, we can use the default settings. For an example of non-English tokenization, see Melanie Walsh, *Introduction to Cultural Analytics & Python*, Version 1 (2021), https://doi.org/10.5281/zenodo.4411250.

In [11]:
example_tool_data['tokenized_example_text'] = example_tool_data['example_text'].apply(word_tokenize)
example_tool_data['tokenized_string_matching'] = example_tool_data['tokenized_example_text'].apply(lambda x: sum(1 for token in x if token in ['tool', 'Tool', 'TOOL']))
example_tool_data.style.pipe(make_pretty, subset_columns=['string_matching', 'tokenized_string_matching']) 

,example_text,string_matching,tokenized_example_text,tokenized_string_matching
0,digital tools,1,"['digital', 'tools']",0
1,Critical Tool Studies,1,"['Critical', 'Tool', 'Studies']",1
2,Footstool,1,['Footstool'],0
3,DH TOOLSETS,1,"['DH', 'TOOLSETS']",0
4,tooling up,1,"['tooling', 'up']",0


Here we have tokenized our `example_text` using the `NLTK` library, rather than `str.split(' ')` since we use NLTK in our article (but they are essentially the same here). We can see that in this example we are now only getting the exact match for `tool`, and not any of other examples. This approach is much more restrictive than string matching. We could add `tools` to our list of allowed terms to get `digital tools`, though then we would need to write `tools` and `Tools` and `TOOLS` to be equally comprehensive.

#### Method 3: Lowercasing

Instead of having to write out all versions of our terms, we can lowercase all of our text to help normalize our data. This will help us avoid the problem of case sensitivity. We can do this with the `str.lower()` method. This type of transformation is often part of pre-processing or data cleaning, but can be enormously impactful on the results of your analysis. However, in our case, we want to capture both `Tool` and `tool`, as well as `Tools` and `tools` so this approach makes sense.

In [17]:
example_tool_data['lower_example_text'] = example_tool_data['example_text'].str.lower()
example_tool_data['lower_string_matching'] = example_tool_data['lower_example_text'].str.count('tool|Tool|TOOL')
example_tool_data['tokenized_lower_example_text'] = example_tool_data['lower_example_text'].apply(word_tokenize)
example_tool_data['tokenized_string_matching'] = example_tool_data['tokenized_example_text'].apply(lambda x: sum(1 for token in x if token in ['tool', 'tools']))
example_tool_data['tokenized_lower_string_matching'] = example_tool_data['tokenized_lower_example_text'].apply(lambda x: sum(1 for token in x if token in ['tool', 'tools']))
example_tool_data[['example_text', 'string_matching', 'tokenized_string_matching', 'lower_string_matching', 'tokenized_lower_string_matching']].style.pipe(make_pretty, subset_columns=['string_matching', 'tokenized_string_matching', 'lower_string_matching', 'tokenized_lower_string_matching']) 


,example_text,string_matching,tokenized_string_matching,lower_string_matching,tokenized_lower_string_matching
0,digital tools,1,1,1,1
1,Critical Tool Studies,1,0,1,1
2,Footstool,1,0,1,0
3,DH TOOLSETS,1,0,1,0
4,tooling up,1,0,1,0


In our tokenization, this time we are searching for both `tool` and `tools`, but we've also tried lowercasing our data. While we can see that the lowercased text is equally as permissive on string matching, but with tokenization we finally get our two instances of `tool` that we want to include. This is because we are now searching for `tool` and `tools` in our tokenized text, rather than just `tool`. We are not getting `Footstool` anymore, but we are also not getting `TOOLSETS` or `tooling`. In our article, we have decided to be a bit more restrictive and just focus on those most obvious instances of tool, but there are methods to get more of these examples if you want to be more inclusive.

#### Method 4: Lemmatization & Stemming

The main other methods for normalizing textual data are lemmatizing and stemming. Lemmatizing and stemming are both methods of reducing words to their root form. Lemmatizing is more sophisticated than stemming, but both are useful for reducing the number of unique words in your dataset. For example, `tools` and `tool` would both be reduced to `tool`. Whereas stemming is a bit more aggressive and would also lower case the word, so `Tools` would also be reduced to `tool`. We can do this with the `nltk.stem` library.

In [18]:
# Initialize stemmer and lemmatizer
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

# Define functions for stemming and lemmatization
def stem_text(tokens):
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)

def lemmatize_text(tokens):
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return ' '.join(lemmatized_tokens)

In [20]:
# Apply the functions to the 'example_text' column
example_tool_data['stemmed_text'] = example_tool_data['tokenized_example_text'].apply(stem_text)
example_tool_data['lemmatized_text'] = example_tool_data['tokenized_example_text'].apply(lemmatize_text)
example_tool_data[['example_text', 'stemmed_text', 'lemmatized_text']]

,example_text,stemmed_text,lemmatized_text
0,digital tools,digit tool,digital tool
1,Critical Tool Studies,critic tool studi,Critical Tool Studies
2,Footstool,footstool,Footstool
3,DH TOOLSETS,dh toolset,DH TOOLSETS
4,tooling up,tool up,tooling up


Here we can see that `stemming` took terms like `digital tools` and turned them into `digit tool`, or in the case of `Critical Tool Studies` it turned it into `critic tool studi`. However, we aren't seeing much changes with the lemmatizing.

In [21]:
# Apply the functions to the 'example_text' column
example_tool_data['stemmed_text'] = example_tool_data['tokenized_lower_example_text'].apply(stem_text)
example_tool_data['lemmatized_text'] = example_tool_data['tokenized_lower_example_text'].apply(lemmatize_text)
example_tool_data[['example_text', 'stemmed_text', 'lemmatized_text']]

,example_text,stemmed_text,lemmatized_text
0,digital tools,digit tool,digital tool
1,Critical Tool Studies,critic tool studi,critical tool study
2,Footstool,footstool,footstool
3,DH TOOLSETS,dh toolset,dh toolsets
4,tooling up,tool up,tooling up


Now we are running both methods on our `lowercased` and `tokenized` data. This time `lemmatizing` is turning `tools` into `tool`, and `studies` into `study`. Whereas `stemming` is not only transforming those terms, but also turning `toolsets` into `toolset` and `tooling` into `tool`.

In [24]:
example_tool_data['tokenized_lemmatized_text'] = example_tool_data['lemmatized_text'].apply(word_tokenize)
example_tool_data['tokenized_lemmatized_string_matching'] = example_tool_data['tokenized_lemmatized_text'].apply(lambda x: sum(1 for token in x if token in ['tool', 'tools']))
example_tool_data['tokenized_stemmed_text'] = example_tool_data['stemmed_text'].apply(word_tokenize)
example_tool_data['tokenized_stemmed_string_matching'] = example_tool_data['tokenized_stemmed_text'].apply(lambda x: sum(1 for token in x if token in ['tool', 'tools']))
example_tool_data[['example_text', 'tokenized_string_matching', 'tokenized_lower_string_matching', 'tokenized_lemmatized_string_matching', 'tokenized_stemmed_string_matching']].style.pipe(make_pretty, subset_columns=['tokenized_string_matching', 'tokenized_lower_string_matching','tokenized_lemmatized_string_matching', 'tokenized_stemmed_string_matching'])

,example_text,tokenized_string_matching,tokenized_lower_string_matching,tokenized_lemmatized_string_matching,tokenized_stemmed_string_matching
0,digital tools,1,1,1,1
1,Critical Tool Studies,0,1,1,1
2,Footstool,0,0,0,0
3,DH TOOLSETS,0,0,0,0
4,tooling up,0,0,0,1


Now we can see that if we rerun our tokenization and string matching code, `lemmatization` gets us similar results to simply lowercasing our data. Whereas `stemming` also gets the example of `tooling` that we were missing before. While we could use `stemming` in our article, we have decided to primarily use `lowercasing` and `tokenization`, along with `string matching` to balance both inclusivity and accuracy.

#### Method 5: Our Article's Approach

Our final added transformation in our article is to not only count words, but to normalize those counts based on the length of their respective document. This helps us know if a term like `tool` is appearing more frequently because it is a longer document, or because it is actually more frequent. We can do this with the `str.len()` method.

In [26]:
example_tool_data['total_length'] = example_tool_data['example_text'].apply(len)
example_tool_data['total_words'] = example_tool_data['tokenized_example_text'].apply(len)
example_tool_data[['example_text', 'total_length', 'total_words']].style.pipe(make_pretty, subset_columns=['total_length', 'total_words'])

,example_text,total_length,total_words
0,digital tools,13,2
1,Critical Tool Studies,21,3
2,Footstool,9,1
3,DH TOOLSETS,11,2
4,tooling up,10,2


We can see that counting simply characters versus words gives us very different results. In our article, we have primarily counted words (or `tokens`) though again this approach is not perfect for every language.

In [32]:
example_tool_data['scaled_tokenized_lower_string_matching'] = example_tool_data['tokenized_lower_string_matching'] / example_tool_data['total_words']
example_tool_data['scaled_percent'] = example_tool_data['scaled_tokenized_lower_string_matching'] * 100

example_tool_data[['example_text', 'tokenized_lower_string_matching', 'total_words', 'scaled_tokenized_lower_string_matching', 'scaled_percent']].style.pipe(make_pretty, subset_columns=['tokenized_lower_string_matching', 'total_words', 'scaled_tokenized_lower_string_matching', 'scaled_percent'])

,example_text,tokenized_lower_string_matching,total_words,scaled_tokenized_lower_string_matching,scaled_percent
0,digital tools,1,2,0.500000,50.000000
1,Critical Tool Studies,1,3,0.333333,33.333333
2,Footstool,0,1,0.000000,0.000000
3,DH TOOLSETS,0,2,0.000000,0.000000
4,tooling up,0,2,0.000000,0.000000


Now we have our final results, which we have used in our article. We have decided to use `lowercasing` and `tokenization` to get our words, and then `string matching` to get our counts. We have also decided to normalize our counts by the length of the document and then finally we have turned those scaled results (which are very small) into percentages. This helps us compare across documents and see which terms are most frequent in each document.